In [1]:
import os ,json

In [ ]:
#You can use python-dotenv module for reading environment variable set inside base environment(local)
# from dotenv import load_dotenv 
# load_dotenv()


In [2]:
 #Here you dont need dotenv because you are loading from local file not from base environment (environment variable)